In [ ]:
# !python3 -m venv analysis-venv
!source analysis-venv/bin/activate


In [ ]:
# pip install --upgrad pandas

In [ ]:
!pip install numpy==1.26.4

In [ ]:
import os
import numpy as np
import joblib

os.getcwd()

In [ ]:
!ls

In [ ]:
!cd analysis

In [ ]:
os.getcwd()

In [ ]:
# %pip uninstall scikit-learn imbalanced-learn
%pip install scikit-learn==1.3.0 imbalanced-learn==0.11.0
!pip install emoji


In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from IPython.display import display
import re
import emoji


In [ ]:
model_dir_name = 'flan-t5-base'

In [ ]:
random_state = 42
np.random.seed(random_state)


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_prediction_explanation_llama = pd.read_csv(file_name)
# df_prediction_explanation_llama=df_prediction_explanation_llama.iloc[:25000]

df_prediction_explanation_llama = df_prediction_explanation_llama.rename(columns={
    'toxicity_score': 'toxicity_score_llama',
    'toxicity_explanation':'toxicity_explanation_llama'
})

file_path = '/path/to/input/files' % replace with real path
df_prediction_explanation_qwen = pd.read_csv(file_name)
# df_prediction_explanation_qwen=df_prediction_explanation_qwen.iloc[:25000]
df_prediction_explanation_qwen = df_prediction_explanation_qwen.rename(columns={
    'issue_id':'issue_url',
    'toxicity_score': 'toxicity_score_qwen',
    'toxicity_explanation':'toxicity_explanation_qwen'
})

df_prediction_explanation = pd.merge(
    df_prediction_explanation_llama[['issue_url', 'toxicity_score_llama', 'toxicity_explanation_llama']],
    df_prediction_explanation_qwen[['issue_url', 'toxicity_score_qwen', 'toxicity_explanation_qwen']],
    on='issue_url')

df_prediction_explanation['is_toxic_llm_pred_llama']= df_prediction_explanation['toxicity_score_llama']>=0.3
df_prediction_explanation['is_toxic_llm_pred_qwen']= df_prediction_explanation['toxicity_score_qwen']>=0.3

df_prediction_explanation['avg_toxicity_score'] = (df_prediction_explanation['toxicity_score_llama']+df_prediction_explanation['toxicity_score_qwen'])/2

df_prediction_explanation['toxicity_score_diff'] = abs(df_prediction_explanation['toxicity_score_llama']-df_prediction_explanation['toxicity_score_qwen'])


df_prediction_explanation['issue_url'] = df_prediction_explanation['issue_url'].astype(str)

df_prediction_explanation



In [ ]:
print(df_prediction_explanation.columns)

In [ ]:

# Filter the DataFrame based on the given conditions
filtered_df = df_prediction_explanation[
    (df_prediction_explanation['toxicity_score_llama'].isnull()) |  # Check for None/NaN
    (df_prediction_explanation['toxicity_score_llama'].apply(lambda x: len(x) > 1 if isinstance(x, str) else False)) |  # Length > 1
    (df_prediction_explanation['toxicity_score_llama'] == "")|
    (df_prediction_explanation['toxicity_score_qwen'].isnull()) |  # Check for None/NaN
    (df_prediction_explanation['toxicity_score_qwen'].apply(lambda x: len(x) > 1 if isinstance(x, str) else False)) |  # Length > 1
    (df_prediction_explanation['toxicity_score_qwen'] == "")  # Empty string
]

# Extract the 'issue_url' values
invalid_issue_urls = filtered_df['issue_url']

# Display the resulting issue URLs
invalid_issue_urls.tolist()

In [ ]:
len(invalid_issue_urls)

In [ ]:
second_person_pronouns = ['you', 'your', 'yours', 'yourself', 'yourselves']
def count_second_person_pronouns(text):
    count = 0
    words = text.lower().split()
    for pronoun in second_person_pronouns:
        count += words.count(pronoun)
    return count

In [ ]:
file_path = '/path/to/input/files' % replace with real path

df_comment_data = pd.read_csv(file_name)

df_comment_data

In [ ]:
print(len(set(df_comment_data['url'])))
print(len(set(df_comment_data['issue_url'])))

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_conversation = pd.read_csv(file_name)
df_conversation

In [ ]:
df_comment_data['body'] = df_comment_data['body'].astype(str).replace({'nan': '', '': ''})
df_comment_data['body'] = df_comment_data['body'].astype(str).fillna('')
df_comment_data['pronoun_count'] = df_comment_data['body'].apply(count_second_person_pronouns)
df_comment_data['comment_length'] = df_comment_data['body'].apply(len)


In [ ]:

# First, group by 'issue_url' and 'speaker_name' and count the comments per speaker
speaker_comment_counts = df_comment_data.groupby(['issue_url', 'user']).size()
speaker_comment_counts

In [ ]:
max_comments_per_issue_by_speakers = speaker_comment_counts.groupby(level=0).max()
max_comments_per_issue_by_speakers

In [ ]:
# Define a function to count '@' in each comment
def count_at_signs(text):
    text = text or ''
    
    try:
        return text.count('@')
    except Exception as e:
        # Catch all other exceptions
        # print("An error occurred:", e)
        return 0
    
def count_quotes(text):
    try:
        return text.count("'") + text.count('"') + text.count('`')
    except Exception as e:
        # print("An error occurred:", e)
        return 0
    
def count_hashes(text):
    try:
        return text.count("#")
    except Exception as e:
        # print("An error occurred:", e)
        return 0

In [ ]:
def count_words(text):
    return len(str(text).split())


# Get max number of comments per speaker per issue
median_comments_per_issue_by_speakers = speaker_comment_counts.groupby(level=0).median()

# Group by 'issue_url' and aggregate
aggregated_data = df_comment_data.groupby('issue_url').agg(
    comment_counts=('url', 'count'),  # Count of comments
    unique_speakers=('user', 'nunique'),  # Count of unique speaker names
    total_second_person_pronouns=('pronoun_count', 'sum'),
    max_comment_length=('comment_length', 'max'),  # Maximum length of comments,
    total_ats=('body', lambda x: x.apply(count_at_signs).sum()),  # Sum of '@' in all comments
    total_quotes=('body', lambda x: x.apply(count_quotes).sum()),  # Sum of quote characters in all comments
    median_words_in_comment=('body', lambda x: x.apply(count_words).median()),
    std_dev_words_in_comment=('body', lambda x: x.apply(count_words).std()),
    max_words_in_comment=('body', lambda x: x.apply(count_words).max())
)

aggregated_data['max_comments_by_one_speaker'] = aggregated_data.index.map(max_comments_per_issue_by_speakers)
aggregated_data['median_comments_by_one_speaker'] = aggregated_data.index.map(median_comments_per_issue_by_speakers)


# Reset index to make 'issue_url' a column again
aggregated_data.reset_index(inplace=True)
# aggregated_data=aggregated_data[:25000]

# Print the result
aggregated_data


In [ ]:
def check_is_closed(group):
    last_two_comments = group.tail(2)  # Get the last 2 comments for the issue_url
    return int(last_two_comments['body'].str.contains(r'\bclose\b', case=False).any())

# Group by issue_url and apply the function
aggregated_data2 = df_comment_data.groupby('issue_url').apply(
    lambda group: pd.Series({'is_issue_closed': check_is_closed(group)})
).reset_index()


def count_emojis(text):
    """
    Count the total number of emojis in a text string.
    
    Args:
        text (str): The text string to analyze (e.g., GitHub issue comment)
    
    Returns:
        int: Total number of emojis found
    """
    # Get all emojis present in the text
    emoji_list = [c for c in text if c in emoji.EMOJI_DATA]
    
    # Return the count
    return len(emoji_list)

def calculate_emoji_metrics(group):
    # Calculate total emojis across all comments for an issue_url
    total_emojis = group['body'].apply(count_emojis).sum()
    
    # Count the number of comments containing at least one emoji
    total_comments_with_emoji = group['body'].apply(lambda x: count_emojis(x) > 0).sum()
    total_comments = len(group)

    return pd.Series({
        'emoji_count': total_emojis,
        'total_comment_with_emoji': total_comments_with_emoji,
        'total_comment_with_emoji_ratio': total_comments_with_emoji/total_comments
    })

# Group by issue_url and apply the function
emoji_metrics = df_comment_data.groupby('issue_url').apply(calculate_emoji_metrics).reset_index()

# Merge with the aggregated_data2
aggregated_data2 = pd.merge(aggregated_data2, emoji_metrics, on='issue_url')

# aggregated_data2['code_of_conduct_mentioned'] = df_comment_data.groupby('issue_url')['text'].apply(
#     lambda comments: int(comments.str.contains("code of conduct", case=False).any())
# ).reset_index(drop=True)

def get_first_coc_mention_idx(comments):
    # Find the first occurrence of "code of conduct" in comments
    mask = comments.str.contains("code of conduct", case=False)
    # If found, return the index (starting from 1), else return 0
    if mask.any():
        return mask.idxmax() + 1  # Adding 1 to convert from 0-based to 1-based indexing
    return 0

# Apply the function to get the index of first code of conduct mention
aggregated_data2['code_of_conduct_mentioned_comment_idx'] = df_comment_data.groupby('issue_url')['body'].apply(
    get_first_coc_mention_idx
).reset_index(drop=True)

aggregated_data2

In [ ]:
# Technical features
def has_stack_trace(text):
    st_regex = re.compile(r'at [a-zA-Z0-9\.<>$]+\(.+\)')
    return bool(st_regex.search(text))

def check_template(first_comment):
    template_indicators = ['### Description', '## Description', '### Steps to reproduce', '## Steps to reproduce']
    return int(any(indicator in first_comment for indicator in template_indicators))

# Update aggregation
def calculate_technical_features(group):
    texts = group['body'].astype(str)
    first_comment = texts.iloc[0] if not texts.empty else ""
    
    return pd.Series({
        'has_stack_trace': int(texts.apply(has_stack_trace).any()),
        'has_code': int(texts.str.contains(r'```\w*\n.*?\n```', flags=re.DOTALL).any()),
        # 'has_markdown': int(texts.str.contains(r'```').any()),
        'has_template': check_template(first_comment)
    })

technical_feature_metrics = df_comment_data.groupby('issue_url').apply(calculate_technical_features).reset_index()
aggregated_data2 = pd.merge(aggregated_data2, technical_feature_metrics, on='issue_url')

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_time_features = pd.read_csv(file_name)

# df_time_features=df_time_features[:25000]
df_time_features

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_text_descriptive = pd.read_csv(file_name)

# df_text_descriptive = df_text_descriptive.drop(columns=['is_toxic','body'])
df_text_descriptive = df_text_descriptive.rename(columns={col:  col+'_TD' if col != 'issue_url' else col for col in df_text_descriptive.columns})

# df_text_descriptive=df_text_descriptive[:25000]
df_text_descriptive

In [ ]:
x= df_text_descriptive['symbol_to_word_ratio_#_TD'].tolist()
x.sort()
x[-5:]

In [ ]:
print(df_text_descriptive['symbol_to_word_ratio_#_TD'].tolist().sort())

In [ ]:
df_text_descriptive.loc[df_text_descriptive['symbol_to_word_ratio_#_TD'] < 0.0001, 'symbol_to_word_ratio_#_TD'] = 0.0001


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_text_content_features= pd.read_csv(file_name)

df_text_content_features=df_text_content_features.drop(columns=['conversation'])

# df_text_content_features=df_text_content_features[:25000]
df_text_content_features

In [ ]:
emotions= [
            # 'admiration', 'amusement',
    'anger', 'annoyance', 'approval',
    # 'caring',
            'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
            'disgust', 'embarrassment', 
    # 'excitement', 'fear', 'gratitude', 'grief',
            # 'joy', 'love', 'nervousness', 'neutral', 'optimism', 
    'pride',
    # 'realization', 'relief', 'remorse', 
    'sadness', 'surprise'
        ]

file_path = '/path/to/input/files' % replace with real path
df_emotion_scores = pd.read_csv(file_path)

df_emotion_scores= df_emotion_scores[emotions+['issue_url']]
# df_emotion_scores=df_emotion_scores[:25000]
df_emotion_scores

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_senticr = pd.read_csv(file_path)
df_sentiment_polarity_senticr

In [ ]:
df_sentiment_polarity_senticr=df_sentiment_polarity_senticr[['issue_url', 'has_neg_comment_sentcr', 'non_neg_comment_ratio_sentcr',
       'neg_comment_ratio_sentcr', 'sentiment_transition_ratio_sentcr']]


In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_textblob = pd.read_csv(file_path)

# df_sentiment_polarity_textblob=df_sentiment_polarity_textblob[:25000]
df_sentiment_polarity_textblob

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_sentiment_polarity_vader = pd.read_csv(file_path)

# df_sentiment_polarity_vader=df_sentiment_polarity_vader[:25000]
df_sentiment_polarity_vader

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_conv_exp = pd.read_csv(file_path)

# df_explanation_features_given_conv_exp=df_explanation_features_given_conv_exp[:25000]
df_explanation_features_given_conv_exp

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_conv = pd.read_csv(file_path)

# df_explanation_features_given_conv=df_explanation_features_given_conv[:25000]
df_explanation_features_given_conv

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_features_given_exp = pd.read_csv(file_path)

# df_explanation_features_given_exp=df_explanation_features_given_exp[:25000]
df_explanation_features_given_exp

In [ ]:
file_path = '/path/to/input/files' % replace with real path
df_explanation_logits_features = pd.read_csv(file_path)

df_explanation_logits_features

In [ ]:

df_explanation_features_given_conv_exp = df_explanation_features_given_conv_exp.rename(columns=
                                                 {
                                                     'toxic_label_probability': 'toxic_prob_conv_exp',
                                                     'non_toxic_label_probability': 'non_toxic_prob_conv_exp',
                                                 })
df_explanation_features_given_conv_exp

In [ ]:

df_explanation_features_given_conv = df_explanation_features_given_conv.rename(columns=
                                     {
                                         'toxic_label_probability': 'toxic_prob_conv',
                                         'non_toxic_label_probability': 'non_toxic_prob_conv',
                                     })
df_explanation_features_given_conv

In [ ]:

df_explanation_features_given_exp = df_explanation_features_given_exp.rename(columns=
                                     {
                                         'toxic_label_probability': 'toxic_prob_exp',
                                         'non_toxic_label_probability': 'non_toxic_prob_exp',
                                     })
df_explanation_features_given_exp

In [ ]:
df_explanation_logits_features = df_explanation_logits_features.rename(columns=
                                     {
                                         'first_token_logit': 'exp_first_token_logit',
                                         'first_token_entropy': 'exp_first_token_entropy',
                                         'avg_entropy': 'exp_avg_entropy',
                                         'avg_logits': 'exp_avg_logits'
                                     })
df_explanation_logits_features

In [ ]:
df_explanation_features = pd.DataFrame()
df_explanation_features = pd.merge(df_explanation_features_given_conv_exp, df_explanation_features_given_conv, on='issue_url')
df_explanation_features = pd.merge(df_explanation_features, df_explanation_features_given_exp, on='issue_url')
df_explanation_features = pd.merge(df_explanation_features, df_explanation_logits_features, on='issue_url')
df_explanation_features=df_explanation_features.drop('Unnamed: 0', axis=1, errors='ignore')
df_explanation_features



In [ ]:
df_explanation_features.columns

In [ ]:
##########################################
df_merged = aggregated_data2.copy()

df_merged = pd.merge(df_merged, df_prediction_explanation[[
    'issue_url', 'toxicity_score_llama', 'is_toxic_llm_pred_llama',
    'toxicity_score_qwen', 'is_toxic_llm_pred_qwen',
    'avg_toxicity_score', 'toxicity_score_diff'
]], on='issue_url')

# df_merged = pd.merge(df_merged, df_bert_score, on='issue_url')
df_merged = pd.merge(df_merged, df_time_features, on='issue_url')

# df_merged = pd.merge(df_merged, df_perplexity, on='issue_url')
df_merged = pd.merge(df_merged, aggregated_data, on='issue_url')

# df_merged = pd.merge(df_merged, df_tone_features, on='issue_url')

# # we may not use emotion scores
df_merged = pd.merge(df_merged, df_emotion_scores, on='issue_url')

df_merged = pd.merge(df_merged, df_sentiment_polarity_senticr, on='issue_url')
df_merged = pd.merge(df_merged, df_sentiment_polarity_textblob, on='issue_url')
df_merged = pd.merge(df_merged, df_sentiment_polarity_vader, on='issue_url')

df_merged = pd.merge(df_merged, df_text_descriptive, on='issue_url')
df_merged = pd.merge(df_merged, df_text_content_features, on='issue_url')

# df_merged = pd.merge(df_merged, df_label_logits_features, on=['issue_url'])

# df_merged = pd.merge(df_merged, df_outlier_score, on='issue_url')

# # df_merged = pd.merge(df_merged, df_explanation_features[['is_toxic', 'issue_url']], on=['issue_url'])
df_merged = pd.merge(df_merged, df_explanation_features, on=['issue_url'])
# df_merged["is_toxic"] = df_explanation_features['is_toxic']


# df_merged
df_merged = df_merged.reset_index(drop=True)

df_merged



In [ ]:
df_merged.columns.tolist()

## Remove Used Conversations

In [ ]:
# df_merged = df_merged[~df_merged['issue_url'].isin(new_labeled_urls)]
# df_merged

### Removing Issues for which LLama Could not Produce Label

In [ ]:
df_merged = df_merged[~df_merged['issue_url'].isin(invalid_issue_urls)]

# Use Same Features as the Model

In [ ]:
feature_columns_df =  pd.read_csv(file_name)
feature_columns = feature_columns_df['features'].to_list()
feature_columns

In [ ]:
len(feature_columns)

In [ ]:
df_merged = df_merged[['issue_url']+feature_columns]

In [ ]:
# df_merged.to_csv("non_labelled_all_features.csv", index = False)

In [ ]:
df_merged.columns

In [ ]:
# Assuming `series` is your Pandas Series
indices_with_none1 = df_merged['is_toxic_llm_pred_llama'][df_merged['is_toxic_llm_pred_llama'].isna()].index.tolist()
indices_with_none2 = df_merged['is_toxic_llm_pred_qwen'][df_merged['is_toxic_llm_pred_qwen'].isna()].index.tolist()

# Display the result
indices_with_none = indices_with_none1+indices_with_none2
indices_with_none

In [ ]:
df_merged = df_merged.drop(index=indices_with_none).reset_index(drop=True)
df_merged

In [ ]:
print(len(df_merged.columns))
df_merged.columns

## Removing NAN

In [ ]:
df_merged['is_toxic_llm_pred_llama'] = df_merged['is_toxic_llm_pred_llama'].astype(int)
df_merged['is_toxic_llm_pred_qwen'] = df_merged['is_toxic_llm_pred_qwen'].astype(int)


In [ ]:
print("LLM predicted toxic samples: ", sum(df_merged['is_toxic_llm_pred_llama']))

In [ ]:
# removing rows where llm couldnot predict properly
df_merged = df_merged[df_merged['is_toxic_llm_pred_llama'].isin([0,1])]
df_merged

In [ ]:
nan_columns = df_merged.columns[df_merged.isna().any()].tolist()

print(nan_columns)

In [ ]:
# removing rows with nan values
df_merged=df_merged.dropna(subset=nan_columns)
df_merged

In [ ]:
string_columns = df_merged.select_dtypes(include=['object', 'string']).columns

print("String columns:", list(string_columns))

## Feature Standardization

In [ ]:
# replace infinite values with nan
df_merged[feature_columns] = df_merged[feature_columns].replace([np.inf, -np.inf], np.nan)


In [ ]:
nan_columns = df_merged.columns[df_merged.isna().any()].tolist()

print(nan_columns)

In [ ]:
# Option 1: Drop rows with inf/nan values
df_merged = df_merged.dropna(subset=feature_columns)
df_merged

In [ ]:
string_columns = df_merged.select_dtypes(include=['object', 'string']).columns

print("String columns:", list(string_columns))

### Ramdomly Sample Conversations

In [ ]:
st_scaler = joblib.load('./verifier_randomforest_models/standardScaler_70_30_split_explainable_features_reduced_with_qwen_final.pkl')

df_standardized = pd.DataFrame(st_scaler.fit_transform(df_merged[feature_columns]), columns=feature_columns)
df_standardized = df_standardized.reset_index(drop=True)
df_standardized['issue_url']=df_merged['issue_url'].reset_index(drop=True)
df_standardized

In [ ]:
nan_columns = df_standardized.columns[df_standardized.isna().any()].tolist()

print(nan_columns)

In [ ]:
print(df_standardized[nan_columns].isna().sum())  # Count NaNs in specified columns

In [ ]:
if 'per_word_perplexity_TD' in df_merged.columns: 
    print(df_merged['per_word_perplexity_TD'].std() )# as std is infinite we need to drop it

In [ ]:
if 'symbol_to_word_ratio_#_TD' in df_merged.columns: 
    print("min value:", df_merged['symbol_to_word_ratio_#_TD'].min() )
    print("max value:", df_merged['symbol_to_word_ratio_#_TD'].max() )
    print("median:",df_merged['symbol_to_word_ratio_#_TD'].median() )
    print("std:", df_merged['symbol_to_word_ratio_#_TD'].std() )# as std is infinite we need to drop it 

In [ ]:
df_standardized=df_standardized.drop(columns=nan_columns)

df_standardized

## Load Models

In [ ]:
file_path = '/path/to/input/files' % replace with real path

loaded_model = joblib.load(file_path)


In [ ]:
def predict_with_model(model, X, return_probabilities=True):
    """
    Make predictions using a loaded model
    
    Parameters:
    model: loaded sklearn model
    X (pandas.DataFrame): Features to predict on
    return_probabilities (bool): If True, return probability scores along with predictions
    
    Returns:
    tuple or array: If return_probabilities is True, returns (predictions, probabilities),
                   otherwise returns just predictions
    """
    predictions = model.predict(X)
    if return_probabilities:
        probabilities = model.predict_proba(X)
        return predictions, probabilities[:, 1]
    return predictions


In [ ]:
# Make predictions using ensemble of all models
_, probabilities = predict_with_model(loaded_model, df_standardized[feature_columns])
predictions = probabilities>=0.4
df_merged['is_correct_verifier_pred_probability'] = probabilities
df_merged['is_correct_verifier_pred_label'] = predictions


In [ ]:
for p in probabilities:
    print(p)

In [ ]:
incorrect_df = df_merged[df_merged['is_correct_verifier_pred_label']==False]
total_incorrect = incorrect_df.shape[0]

print(f'Total incorrect prediction by verifier is {total_incorrect} out of {df_merged.shape[0]}')

In [ ]:
print(f"Total llama predicted toxic conversation is {df_merged[df_merged['is_toxic_llm_pred_llama']==True].shape[0]} out of {df_merged.shape[0]}")

In [ ]:
correct_df = df_merged[df_merged['is_correct_verifier_pred_label']==True]

correct_df

In [ ]:
correct_toxic_df = correct_df[correct_df['is_toxic_llm_pred_llama']==True]
print(f"{correct_toxic_df.shape[0]} LLM predicted toxic are  in {correct_df.shape[0]} correct predictions")

In [ ]:
incorrect_df = df_merged[df_merged['is_correct_verifier_pred_label']==False]

incorrect_df = incorrect_df.merge(df_conversation, on='issue_url', how='left')
incorrect_df

In [ ]:
incorrect_df.to_csv('./output/large_dataset_validated_incorrect_conversations_final.csv', index = False)

In [ ]:
correct_df = df_merged[df_merged['is_correct_verifier_pred_label']==True]
correct_df

In [ ]:
correct_df.to_csv('./output/large_dataset_validated_correct_conversations_final.csv', index = False)